In [1]:
import settings
from process_sentinel2 import *

In [2]:
DATA_PATH = settings.data_path
DATE_FORMAT = '%Y-%m-%d'
START_DATE = '2016-12-21'
END_DATE = '2021-04-20'

### Generate data from date range and keep dates of cloudy and non-cloudy days

In [33]:
data_generator = laguna_data_generator(START_DATE, END_DATE, DATE_FORMAT, DATA_PATH)
clear_data = []
cloudy_data = []
for day in data_generator:
    if day == None:
        continue
    if np.std(day.dominant_color()) < 2:
        cloudy_data.append(day.date)
        print("Dia nublado")
    else:
        clear_data.append(day.date)
        print("Dia lindo")

2017-11-04
Dia lindo
2017-11-12
Dia nublado
2017-11-17
Dia lindo
2017-11-19
Dia lindo
2017-11-22
Dia lindo
2017-11-24
2017-11-29
Dia lindo
2017-12-02
Dia nublado
2017-12-07
Dia lindo
2017-12-09
Dia nublado
2017-12-12
Dia nublado
2017-12-14
2017-12-19
Dia nublado
2017-12-22
Dia lindo
2017-12-27
Dia nublado
2017-12-29
Dia lindo
2018-01-01
2018-01-03
2018-01-08
Dia nublado
2018-01-11
Dia lindo
2018-01-16
Dia nublado
2018-01-18
2018-01-21
Dia nublado
2018-01-23
Dia lindo
2018-01-28
Dia nublado
2018-01-31
Dia nublado
2018-02-07
Dia lindo
2018-02-10
Dia lindo
2018-02-12
Dia nublado
2018-02-15
Dia nublado
2018-02-17
Dia lindo
2018-02-20
Dia lindo
2018-02-22
Dia nublado
2018-02-25
Dia nublado
2018-02-27
2018-03-02
Dia lindo
2018-03-04
2018-03-07
Dia lindo
2018-03-09
Dia lindo
2018-03-12
Dia lindo
2018-03-14
Dia lindo
2018-03-17
Dia nublado
2018-03-19
Dia lindo
2018-03-22
Dia lindo
2018-03-24
Dia lindo
2018-03-27
Dia nublado
2018-03-29
Dia lindo
2018-04-01
Dia nublado
2018-04-03
Dia lindo
2018-